** The codes below does sequence predictedWordiction **
1. When Number of LSTM =1,2 ,3
2.The preiction of 50 words when the input sequence length =2,5,10

The below code has 1 layer LSTM, and input sequence = 5
Observations:
The training predictedWordiction accuracy increases with number of iterations 

In [1]:

from __future__ import print_function
import numpy as np
import tensorflow as tf
from tensorflow.contrib import rnn
import random
import collections
import time
import codecs

training_file = 'text-Copy1.txt'
input_file =  'text-Copy1.txt'
with codecs.open(input_file, "r", encoding=None) as f:
    data = f.read()

    
############################ Reading the text file and tokenizing every word ##########################
x_text = data.split()
totalWords = len(x_text)
def read_data(fname):
    with open(fname) as f:
        content = f.readlines()
    content = [x.strip() for x in content]
    content = [content[i].split() for i in range(len(content))]
    content = np.array(content)
    content = np.reshape(content, [-1, ])
    print(content.shape)
    return content

training_data = read_data(training_file)
print("Reading the text")

def vocabularyBuild(words):
    count = collections.Counter(words).most_common()
    wordToInt = dict()
    for word, _ in count:
        wordToInt[word] = len(wordToInt)
    intToWord = dict(zip(wordToInt.values(), wordToInt.input_words()))
    return wordToInt, intToWord

wordToInt, intToWord = vocabularyBuild(training_data)
vocab_size = len(wordToInt)
# Parameters
learning_rate = 0.001
training_iters = 50000
frequency = 5000
n_input = 5

# number of units in RNN cell
n_hidden = 512
# tf Graph input
x = tf.placeholder("float", [None, n_input, 1])
y = tf.placeholder("float", [None, vocab_size])

# The final hidden state of RNN
weights = {'out': tf.Variable(tf.random_normal([n_hidden, vocab_size]))}
biases = {'out': tf.Variable(tf.random_normal([vocab_size]))}

########### defining the RNN Model ######################################################3
def RNN(x, weights, biases):
    x = tf.reshape(x, [-1, n_input])
    x = tf.split(x,n_input,1)
    #Define RNN for a single LSTM cell
    rnn_cell = rnn.MultiRNNCell([rnn.BasicLSTMCell(n_hidden)])
    outputs, states = rnn.static_rnn(rnn_cell, x, dtype=tf.float32)
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

predictedWord = RNN(x, weights, biases)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predictedWord, labels=y))
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)

correct_predictedWord = tf.equal(tf.argmax(predictedWord,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_predictedWord, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()
with tf.Session() as session:
    session.run(init)
    step = 0
    offset = random.randint(0,n_input+1)
    end_offset = n_input + 1
    acc_total = 0
    loss_total = 0
    while step < training_iters:
        if offset > (len(training_data)-end_offset):
            offset = random.randint(0, n_input+1)
        #At every iteration a random index is chosen and n_input words are chosen for input
        inputSequence = [ [wordToInt[ str(training_data[i])]] for i in range(offset, offset+n_input) ]
        inputSequence = np.reshape(np.array(inputSequence), [-1, n_input, 1])
        #Encode the input sequence to integers
        encodedOutput = np.zeros([vocab_size], dtype=float)
        encodedOutput[wordToInt[str(training_data[offset+n_input])]] = 1.0
        encodedOutput = np.reshape(encodedOutput,[1,-1])
        opt, acc, loss, onehot_pred = session.run([optimizer, accuracy, cost, predictedWord], \
                                                feed_dict={x: inputSequence, y: encodedOutput})
        loss_total += loss
        acc_total += acc
        if (step+1) % frequency == 0:
            print("Iter= " + str(step+1) + "Training Accuracy= " + "{:.2f}%".format(100*acc_total/frequency))
            acc_total = 0
            loss_total = 0
            inSeq = [training_data[i] for i in range(offset, offset + n_input)]
            outSeq = training_data[offset + n_input]
            predictedSeq = intToWord[int(tf.argmax(onehot_pred, 1).eval())]
        step += 1
        offset += (n_input+1)
    print("Training Over for 1-layer LSTM")
    start_index = random.randint(1,1900)
    sentence = 'following a probe by the'
    sentence = sentence.strip()
    words = sentence.split(' ')
    inputSequence = [wordToInt[words[i]] for i in range(len(words))]
    for i in range(50):
#         print(i)
        input_words = np.reshape(np.array(inputSequence), [-1, n_input, 1])
        onehot_pred = session.run(predictedWord, feed_dict={x: input_words})
        predictedWordIndex = int(tf.argmax(onehot_pred, 1).eval())
        sentence = "%s %s" % (sentence,intToWord[predictedWordIndex])
        inputSequence = inputSequence[1:]
        inputSequence.append(predictedWordIndex)
        nw = sentence.split(' ')
        if len(nw)>50:
            print("Output Random sentence",sentence)
            break


(2041,)
Reading the text
Iter= 5000Training Accuracy= 3.06%
Iter= 10000Training Accuracy= 6.62%
Iter= 15000Training Accuracy= 16.38%
Iter= 20000Training Accuracy= 28.14%
Iter= 25000Training Accuracy= 24.22%
Iter= 30000Training Accuracy= 42.88%
Iter= 35000Training Accuracy= 49.46%
Iter= 40000Training Accuracy= 45.54%
Iter= 45000Training Accuracy= 49.24%
Iter= 50000Training Accuracy= 53.24%
Training Over for 1-layer LSTM
Output Random sentence following a probe by the us securities and downfall the amount shares continuing shares . the to full has but to it also reported basis shares to growth % $amount was 123 performance to $amount at has half to a creditors owns to year earlier action to sale , had has


2) The below code has 2 layer LSTM with, input sequence =5
As expected, the accuracy during training is higher compared to 1 layer LSTM 

In [1]:
#2 layer
from __future__ import print_function
import numpy as np
import tensorflow as tf
from tensorflow.contrib import rnn
import random
import collections
import time
import codecs

start_time = time.time()
training_file = 'text-Copy1.txt'
input_file =  'text-Copy1.txt'
with codecs.open(input_file, "r", encoding=None) as f:
    data = f.read()

x_text = data.split()
totalWords = len(x_text)
def read_data(fname):
    with open(fname) as f:
        content = f.readlines()
    content = [x.strip() for x in content]
    content = [content[i].split() for i in range(len(content))]
    content = np.array(content)
    content = np.reshape(content, [-1, ])
    print(content.shape)
    return content

training_data = read_data(training_file)
print("Reading the text")

def vocabularyBuild(words):
    count = collections.Counter(words).most_common()
    wordToInt = dict()
    for word, _ in count:
        wordToInt[word] = len(wordToInt)
    intToWord = dict(zip(wordToInt.values(), wordToInt.input_words()))
    return wordToInt, intToWord

wordToInt, intToWord = vocabularyBuild(training_data)
vocab_size = len(wordToInt)
# Parameters
learning_rate = 0.001
training_iters = 50000
frequency = 5000
n_input = 5

# number of units in RNN cell
n_hidden = 512
# tf Graph input
x = tf.placeholder("float", [None, n_input, 1])
y = tf.placeholder("float", [None, vocab_size])

# The final hidden state of RNN
weights = {'out': tf.Variable(tf.random_normal([n_hidden, vocab_size]))}
biases = {'out': tf.Variable(tf.random_normal([vocab_size]))}

def RNN(x, weights, biases):
    x = tf.reshape(x, [-1, n_input])
    x = tf.split(x,n_input,1)
    #Define RNN for a single LSTM cell
    rnn_cell = rnn.MultiRNNCell([rnn.BasicLSTMCell(n_hidden),rnn.BasicLSTMCell(n_hidden)])
    outputs, states = rnn.static_rnn(rnn_cell, x, dtype=tf.float32)
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

predictedWord = RNN(x, weights, biases)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predictedWord, labels=y))
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)

correct_predictedWord = tf.equal(tf.argmax(predictedWord,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_predictedWord, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()
with tf.Session() as session:
    session.run(init)
    step = 0
    offset = random.randint(0,n_input+1)
    end_offset = n_input + 1
    acc_total = 0
    loss_total = 0
    while step < training_iters:
        if offset > (len(training_data)-end_offset):
            offset = random.randint(0, n_input+1)
        inputSequence = [ [wordToInt[ str(training_data[i])]] for i in range(offset, offset+n_input) ]
        inputSequence = np.reshape(np.array(inputSequence), [-1, n_input, 1])
        encodedOutput = np.zeros([vocab_size], dtype=float)
        encodedOutput[wordToInt[str(training_data[offset+n_input])]] = 1.0
        encodedOutput = np.reshape(encodedOutput,[1,-1])
        opt, acc, loss, onehot_pred = session.run([optimizer, accuracy, cost, predictedWord], feed_dict={x: inputSequence, y: encodedOutput})
        loss_total += loss
        acc_total += acc
        if (step+1) % frequency == 0:
            print("Iter= " + str(step+1) + "Training Accuracy= " + "{:.2f}%".format(100*acc_total/frequency))
            acc_total = 0
            loss_total = 0
            inSeq = [training_data[i] for i in range(offset, offset + n_input)]
            outSeq = training_data[offset + n_input]
            predictedSeq = intToWord[int(tf.argmax(onehot_pred, 1).eval())]
        step += 1
        offset += (n_input+1)
    print("Training Over for 1-layer LSTM")
    start_index = random.randint(1,1900)
    sentence = 'following a probe by the'
    sentence = sentence.strip()
    words = sentence.split(' ')
    inputSequence = [wordToInt[words[i]] for i in range(len(words))]
    for i in range(50):
#         print(i)
        input_words = np.reshape(np.array(inputSequence), [-1, n_input, 1])
        onehot_pred = session.run(predictedWord, feed_dict={x: input_words})
        predictedWordIndex = int(tf.argmax(onehot_pred, 1).eval())
        sentence = "%s %s" % (sentence,intToWord[predictedWordIndex])
        inputSequence = inputSequence[1:]
        inputSequence.append(predictedWordIndex)
        nw = sentence.split(' ')
        if len(nw)>50:
            print("Output Random sentence",sentence)
            break


(2041,)
Reading the text
Iter= 5000Training Accuracy= 4.20%
Iter= 10000Training Accuracy= 4.52%
Iter= 15000Training Accuracy= 14.88%
Iter= 20000Training Accuracy= 16.18%
Iter= 25000Training Accuracy= 15.84%
Iter= 30000Training Accuracy= 23.98%
Iter= 35000Training Accuracy= 33.36%
Iter= 40000Training Accuracy= 42.52%
Iter= 45000Training Accuracy= 59.52%
Iter= 50000Training Accuracy= 61.42%
Training Over for 1-layer LSTM
Output Random sentence following a probe by the us securities deficit is shows up a a $amount drop a an pernod a said friday . rod of a wall deficits google in a is court in an pernod a said friday . rod of a wall deficits google in a is court in an


3) The below code is 3 layer LSTM for 2 input sequence.
 As it can be seen, the training acuracy increases slowly in comparison to above model. The reason could be due to a smaller input sequence due to which the time dependency learnt is negligible.

In [1]:
#3 layer and input length is 2
from __future__ import print_function
import numpy as np
import tensorflow as tf
from tensorflow.contrib import rnn
import random
import collections
import time
import codecs

start_time = time.time()
training_file = 'text-Copy1.txt'
input_file =  'text-Copy1.txt'
with codecs.open(input_file, "r", encoding=None) as f:
    data = f.read()

x_text = data.split()
totalWords = len(x_text)
def read_data(fname):
    with open(fname) as f:
        content = f.readlines()
    content = [x.strip() for x in content]
    content = [content[i].split() for i in range(len(content))]
    content = np.array(content)
    content = np.reshape(content, [-1, ])
    print(content.shape)
    return content

training_data = read_data(training_file)
print("Reading the text")

def vocabularyBuild(words):
    count = collections.Counter(words).most_common()
    wordToInt = dict()
    for word, _ in count:
        wordToInt[word] = len(wordToInt)
    intToWord = dict(zip(wordToInt.values(), wordToInt.input_words()))
    return wordToInt, intToWord

wordToInt, intToWord = vocabularyBuild(training_data)
vocab_size = len(wordToInt)
# Parameters
learning_rate = 0.001
training_iters = 50000
frequency = 5000
n_input = 2

# number of units in RNN cell
n_hidden = 512
# tf Graph input
x = tf.placeholder("float", [None, n_input, 1])
y = tf.placeholder("float", [None, vocab_size])

# The final hidden state of RNN
weights = {'out': tf.Variable(tf.random_normal([n_hidden, vocab_size]))}
biases = {'out': tf.Variable(tf.random_normal([vocab_size]))}

def RNN(x, weights, biases):
    x = tf.reshape(x, [-1, n_input])
    x = tf.split(x,n_input,1)
    #Define RNN for a single LSTM cell
    rnn_cell = rnn.MultiRNNCell([rnn.BasicLSTMCell(n_hidden),rnn.BasicLSTMCell(n_hidden),rnn.BasicLSTMCell(n_hidden)])
    outputs, states = rnn.static_rnn(rnn_cell, x, dtype=tf.float32)
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

predictedWord = RNN(x, weights, biases)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predictedWord, labels=y))
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)

correct_predictedWord = tf.equal(tf.argmax(predictedWord,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_predictedWord, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()
with tf.Session() as session:
    session.run(init)
    step = 0
    offset = random.randint(0,n_input+1)
    end_offset = n_input + 1
    acc_total = 0
    loss_total = 0
    while step < training_iters:
        if offset > (len(training_data)-end_offset):
            offset = random.randint(0, n_input+1)
        inputSequence = [ [wordToInt[ str(training_data[i])]] for i in range(offset, offset+n_input) ]
        inputSequence = np.reshape(np.array(inputSequence), [-1, n_input, 1])
        encodedOutput = np.zeros([vocab_size], dtype=float)
        encodedOutput[wordToInt[str(training_data[offset+n_input])]] = 1.0
        encodedOutput = np.reshape(encodedOutput,[1,-1])
        opt, acc, loss, onehot_pred = session.run([optimizer, accuracy, cost, predictedWord], feed_dict={x: inputSequence, y: encodedOutput})
        loss_total += loss
        acc_total += acc
        if (step+1) % frequency == 0:
            print("Iter= " + str(step+1) + "Training Accuracy= " + "{:.2f}%".format(100*acc_total/frequency))
            acc_total = 0
            loss_total = 0
            inSeq = [training_data[i] for i in range(offset, offset + n_input)]
            outSeq = training_data[offset + n_input]
            predictedSeq = intToWord[int(tf.argmax(onehot_pred, 1).eval())]
        step += 1
        offset += (n_input+1)
    print("Training Over for 3-layer LSTM")
    start_index = random.randint(1,1900)
    sentence = 'dollar higher'
    sentence = sentence.strip()
    words = sentence.split(' ')
    inputSequence = [wordToInt[words[i]] for i in range(len(words))]
    for i in range(50):
#         print(i)
        input_words = np.reshape(np.array(inputSequence), [-1, n_input, 1])
        onehot_pred = session.run(predictedWord, feed_dict={x: input_words})
        predictedWordIndex = int(tf.argmax(onehot_pred, 1).eval())
        sentence = "%s %s" % (sentence,intToWord[predictedWordIndex])
        inputSequence = inputSequence[1:]
        inputSequence.append(predictedWordIndex)
        nw = sentence.split(' ')
        if len(nw)>50:
            print("Output sentence for input of 2 words",sentence)
            break
    
#     words =
    

(2041,)
Reading the text
Iter= 5000Training Accuracy= 4.02%
Iter= 10000Training Accuracy= 5.20%
Iter= 15000Training Accuracy= 5.90%
Iter= 20000Training Accuracy= 7.56%
Iter= 25000Training Accuracy= 7.04%
Iter= 30000Training Accuracy= 7.52%
Iter= 35000Training Accuracy= 8.54%
Iter= 40000Training Accuracy= 13.10%
Iter= 45000Training Accuracy= 11.42%
Iter= 50000Training Accuracy= 14.54%
Training Over for 3-layer LSTM
Output sentence for input of 2 words dollar higher to its main demand , a major in pre owed , a major in pre owed , a major in pre owed , a major in pre owed , a major in pre owed , a major in pre owed , a major in pre owed , a major


4) The below code does predictedWordiction using 3 layer of LSTM for an input sequence of length 10. The training accuracy is very low for 50000 iterations. Also, the generated sentence gives empty string. This could be because the network was unable to learn a long sequence and probably requires more number of iterations. Also, vanishing gradient could have been a problem due to a longer sequence into the past. This problem could be overcome by gradient clipping.

In [1]:
#3 layer and input sequence length 10
from __future__ import print_function
import numpy as np
import tensorflow as tf
from tensorflow.contrib import rnn
import random
import collections
import time
import codecs

start_time = time.time()
training_file = 'text-Copy1.txt'
input_file =  'text-Copy1.txt'
with codecs.open(input_file, "r", encoding=None) as f:
    data = f.read()

x_text = data.split()
totalWords = len(x_text)
def read_data(fname):
    with open(fname) as f:
        content = f.readlines()
    content = [x.strip() for x in content]
    content = [content[i].split() for i in range(len(content))]
    content = np.array(content)
    content = np.reshape(content, [-1, ])
    print(content.shape)
    return content

training_data = read_data(training_file)
print("Reading the text")

def vocabularyBuild(words):
    count = collections.Counter(words).most_common()
    wordToInt = dict()
    for word, _ in count:
        wordToInt[word] = len(wordToInt)
    intToWord = dict(zip(wordToInt.values(), wordToInt.input_words()))
    return wordToInt, intToWord

wordToInt, intToWord = vocabularyBuild(training_data)
vocab_size = len(wordToInt)
# Parameters
learning_rate = 0.001
training_iters = 50000
frequency = 5000
n_input = 10

# number of units in RNN cell
n_hidden = 512
# tf Graph input
x = tf.placeholder("float", [None, n_input, 1])
y = tf.placeholder("float", [None, vocab_size])

# The final hidden state of RNN
weights = {'out': tf.Variable(tf.random_normal([n_hidden, vocab_size]))}
biases = {'out': tf.Variable(tf.random_normal([vocab_size]))}

def RNN(x, weights, biases):
    x = tf.reshape(x, [-1, n_input])
    x = tf.split(x,n_input,1)
    #Define RNN for a single LSTM cell
    rnn_cell = rnn.MultiRNNCell([rnn.BasicLSTMCell(n_hidden),rnn.BasicLSTMCell(n_hidden),rnn.BasicLSTMCell(n_hidden)])
    outputs, states = rnn.static_rnn(rnn_cell, x, dtype=tf.float32)
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

predictedWord = RNN(x, weights, biases)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predictedWord, labels=y))
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)

correct_predictedWord = tf.equal(tf.argmax(predictedWord,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_predictedWord, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()
with tf.Session() as session:
    session.run(init)
    step = 0
    offset = random.randint(0,n_input+1)
    end_offset = n_input + 1
    acc_total = 0
    loss_total = 0
    while step < training_iters:
        if offset > (len(training_data)-end_offset):
            offset = random.randint(0, n_input+1)
        inputSequence = [ [wordToInt[ str(training_data[i])]] for i in range(offset, offset+n_input) ]
        inputSequence = np.reshape(np.array(inputSequence), [-1, n_input, 1])
        encodedOutput = np.zeros([vocab_size], dtype=float)
        encodedOutput[wordToInt[str(training_data[offset+n_input])]] = 1.0
        encodedOutput = np.reshape(encodedOutput,[1,-1])
        opt, acc, loss, onehot_pred = session.run([optimizer, accuracy, cost, predictedWord], feed_dict={x: inputSequence, y: encodedOutput})
        loss_total += loss
        acc_total += acc
        if (step+1) % frequency == 0:
            print("Iter= " + str(step+1) + "Training Accuracy= " + "{:.2f}%".format(100*acc_total/frequency))
            acc_total = 0
            loss_total = 0
            inSeq = [training_data[i] for i in range(offset, offset + n_input)]
            outSeq = training_data[offset + n_input]
            predictedSeq = intToWord[int(tf.argmax(onehot_pred, 1).eval())]
        step += 1
        offset += (n_input+1)
    print("Training Over for 3-layer LSTM")
    start_index = random.randint(1,1900)
    sentence = 'yukos claims its downfall was punishment for the political ambitions'
    sentence = sentence.strip()
    words = sentence.split(' ')
    inputSequence = [wordToInt[words[i]] for i in range(len(words))]
    for i in range(50):
#         print(i)
        input_words = np.reshape(np.array(inputSequence), [-1, n_input, 1])
        onehot_pred = session.run(predictedWord, feed_dict={x: input_words})
        predictedWordIndex = int(tf.argmax(onehot_pred, 1).eval())
        sentence = "%s %s" % (sentence,intToWord[predictedWordIndex])
        inputSequence = inputSequence[1:]
        inputSequence.append(predictedWordIndex)
        nw = sentence.split(' ')
        if len(nw)>50:
            print("Output sentence for input of 10 words",sentence)
            break
    
#     words =
    

(2041,)
Reading the text
Iter= 5000Training Accuracy= 3.58%
Iter= 10000Training Accuracy= 3.66%
Iter= 15000Training Accuracy= 4.10%
Iter= 20000Training Accuracy= 3.90%
Iter= 25000Training Accuracy= 3.08%
Iter= 30000Training Accuracy= 3.14%
Iter= 35000Training Accuracy= 3.00%
Iter= 40000Training Accuracy= 3.08%
Iter= 45000Training Accuracy= 2.64%
Iter= 50000Training Accuracy= 3.12%
Training Over for 3-layer LSTM
Output sentence for input of 10 words yukos claims its downfall was punishment for the political ambitions . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
